# CasADi C++ 接口与用法（纯 C++ 版）

> 目标：只讲 CasADi 在 **C++** 下的接口、用法和工程化注意事项。
- 不包含 Python/MATLAB 示例
- 代码以“可复制到 `.cpp` / CMake 里即可编译”为标准

> 建议阅读顺序：先看 1（工程接入）→ 2/3（SX/MX/DM 与矩阵运算）→ 4/5（Function 与 AD）→ 6/7/8（integrator/nlpsol/Opti）→ 9（codegen/external）→ 10（排错）。

## 1. 工程接入：头文件、链接、运行时插件

### 1.1 最小 include
```cpp
#include <casadi/casadi.hpp>
```

### 1.2 CMake（推荐）
你已经把 CasADi 装到了 `/usr/local`（你之前的环境就是这种），典型写法：
```cmake
cmake_minimum_required(VERSION 3.16)
project(casadi_cpp_demo LANGUAGES CXX)
set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED ON)

find_package(casadi CONFIG REQUIRED)
add_executable(demo main.cpp)
target_link_libraries(demo PRIVATE casadi)
```

如果你用的是 `find_package(casadi REQUIRED)` 也没问题，取决于 CasADi 安装导出的 config。

### 1.3 g++（临时验证）
```bash
g++ main.cpp -std=c++17 $(pkg-config --cflags --libs casadi) -o demo
```

### 1.4 运行时插件/动态库常见问题
CasADi 的求解器/积分器很多是插件化加载：
- `ipopt`（NLP）
- `cvodes/idas`（integrator）
- `osqp/qpoases`（QP）

常见报错：`... Plugin 'ipopt' is not found`。通常原因是：
- 插件库没安装，或
- 动态库搜索路径没包含 CasADi 的安装目录（例如 `/usr/local/lib`），需要 `sudo ldconfig` 或设置 `LD_LIBRARY_PATH`（不推荐长期依赖）。

下面给一个“自检”最小 main：打印版本 + 一个 Jacobian：
```cpp
#include <casadi/casadi.hpp>

int main() {
  std::cout << "CasADi version: " << casadi::CasadiMeta::version() << "\n";
  casadi::MX x = casadi::MX::sym("x");
  casadi::MX J = casadi::jacobian(casadi::sin(x), x);
  std::cout << "d/dx sin(x) = " << J << "\n";
  return 0;
}
```

## 2. 核心类型：SX / MX / DM（C++ 视角）

你可以把它们粗略理解为：
- `SX`：更偏“标量表达式图”（适合小型、可展开的表达）
- `MX`：更通用的“图”（更适合大型 NLP/OCP，能嵌套 Function 调用）
- `DM`：数值矩阵（Dense Matrix），用于数值输入输出、初值、解

### 2.1 创建与打印

```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::SX xs = casadi::SX::sym("xs");
  casadi::MX xm = casadi::MX::sym("xm", 2, 2);
  casadi::DM A = casadi::DM(std::vector<double>{1,2,3,4}); // 4 x 1
  casadi::DM B = casadi::DM::reshape(A, 2, 2);

  std::cout << "xs=" << xs << "\n";
  std::cout << "xm=" << xm << "\n";
  std::cout << "B=" << B << "\n";
  return 0;
}
```

### 2.2 选择建议（经验法则）
- 先用 `MX` 把问题搭起来（尤其是带约束/求解器的优化问题）。
- 需要极致性能的纯代数/纯 ODE RHS，可用 `SX` 封成 `Function` 再复用。
- 数值数据用 `DM`。

## 3. 常用运算：拼接、转置、矩阵乘法、数学函数（避免命名空间坑）

### 3.1 拼接/转置/reshape
```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::MX x = casadi::MX::sym("x", 3, 1);
  casadi::MX y = casadi::MX::sym("y", 3, 1);

  casadi::MX h = casadi::horzcat(std::vector<casadi::MX>{x, y}); // 3 x 2
  casadi::MX v = casadi::vertcat(std::vector<casadi::MX>{x, y}); // 6 x 1
  casadi::MX ht = casadi::transpose(h);

  std::cout << "h=" << h << "\n";
  std::cout << "v=" << v << "\n";
  std::cout << "ht=" << ht << "\n";
  return 0;
}
```

### 4.0.1 `X(row_slice, col_slice)` 的参数是什么？

CasADi 的矩阵索引（对 `SX/MX/DM` 都成立）一般是：
- `X(i, j)`：取单个元素（第 i 行、第 j 列）
- `X(i, all)` / `X(all, j)`：取某一整行/整列
- `X(row_slice, col_slice)`：取子块（submatrix）
- `X(row_slice, j)`：固定列，行用切片
- `X(i, col_slice)`：固定行，列用切片

其中：
- `i/j` 是 **整数索引**（0 基）。
- `row_slice/col_slice` 是 `casadi::Slice`（可以是 `all` 或 `Slice(start, stop, step)`）。

### 4.1 行/列/子块切片（读切片）

```cpp
#include <casadi/casadi.hpp>
#include <iostream>

int main() {
  casadi::MX X = casadi::MX::sym("X", 2, 6);
  casadi::Slice all;

  // 1) 整行 / 整列
  auto row0 = X(0, all);                 // 1 x 6  第 0 行
  auto col3 = X(all, 3);                 // 2 x 1  第 3 列

  // 2) 连续区间：列 [1,5) -> 第 1,2,3,4 列
  auto mid_cols = X(all, casadi::Slice(1, 5));   // 2 x 4

  // 3) 带步长：列 [0,6) step=2 -> 第 0,2,4 列
  auto even_cols = X(all, casadi::Slice(0, 6, 2)); // 2 x 3

  // 4) 子块：行 [0,2) 与 列 [1,5)
  auto block = X(casadi::Slice(0, 2), casadi::Slice(1, 5)); // 2 x 4

  std::cout << "row0=" << row0 << "\n";
  std::cout << "col3=" << col3 << "\n";
  std::cout << "mid_cols="  << mid_cols  << "\n";
  std::cout << "even_cols=" << even_cols << "\n";
  std::cout << "block="     << block     << "\n";
  return 0;
}
```

## 5. Function：封装表达式 + 调用（vector / DMDict）

`casadi::Function` 可以把一个（或多个）符号表达式封装成“可调用对象”。它有 **明确的输入列表（inputs）** 和 **输出列表（outputs）**：
- 每个 input/output 都有 **位置（index）**，也可以有 **名字（name）**。
- 调用时你既可以按位置传参（`std::vector<DM>`），也可以按名字传参（`DMDict`）。

下面按“构造 → 参数/返回值 → 调用”把最关键的接口讲清楚。

### 5.1 从表达式构造 Function（定义 inputs/outputs 与名字）
```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::MX x = casadi::MX::sym("x", 2, 1);
  casadi::MX y = casadi::MX::sym("y", 2, 1);
  casadi::MX f = casadi::dot(x - y, x - y);

  // Function(name, inputs, outputs, input_names, output_names)
  casadi::Function F("F", {x, y}, {f}, {"x", "y"}, {"f"});
  std::cout << F << "\n";
  return 0;
}
```

### 5.2 输入/输出到底是什么？（参数与返回值详细说明）

#### 5.2.1 输入（inputs）
一个 Function 的输入是“有序列表”，每个输入都有：
- **名字**：`F.name_in(i)`，名字数组 `F.name_in()`
- **形状**（行列数）：`F.size1_in(i)` / `F.size2_in(i)`
- **稀疏模式**：`F.sparsity_in(i)`（决定结构/非零模式）

常用查询：
```cpp
int nin = F.n_in();
for (int i = 0; i < nin; ++i) {
  std::cout << "in[" << i << "] name=" << F.name_in(i)
            << " shape=" << F.size1_in(i) << "x" << F.size2_in(i)
            << " sp=" << F.sparsity_in(i) << "\n";
}
```

> 直觉：inputs 就是你在构造 `Function("F", {x, y}, ...)` 里传进去的 `{x, y}` 那个列表。

#### 5.2.2 输出（outputs）
输出同样是“有序列表”，每个输出都有：
- **名字**：`F.name_out(i)`，名字数组 `F.name_out()`
- **形状**：`F.size1_out(i)` / `F.size2_out(i)`
- **稀疏模式**：`F.sparsity_out(i)`

查询方式与 inputs 完全对应：
```cpp
int nout = F.n_out();
for (int i = 0; i < nout; ++i) {
  std::cout << "out[" << i << "] name=" << F.name_out(i)
            << " shape=" << F.size1_out(i) << "x" << F.size2_out(i)
            << " sp=" << F.sparsity_out(i) << "\n";
}
```

> 直觉：outputs 就是你在构造里传进去的 `{f}` 列表；如果你传 `{f, g, H}`，那 Function 就会有 3 个输出。

#### 5.2.3 形状（shape）与类型（DM/MX/SX）关系
- `Function` 本身是一个“符号计算图的封装”，内部可以基于 `SX` 或 `MX` 构造。
- 但 **调用时的数值输入输出** 最常用的是 `DM`：因为 `DM` 是数值矩阵。
- 因此：你会看到“Function 是 `Function`，参数是 `DM`，返回也是 `DM`（或装在容器里）”。

### 5.3 调用方式 A：`std::vector<DM>`（按位置传参，返回 vector<DM>）

当你使用 `std::vector<DM>` 调用时：
- 输入：按 input 的 **顺序** 填到 vector 里
- 输出：返回 `std::vector<DM>`，按 output 的 **顺序** 排列

```cpp
casadi::DM x0 = casadi::DM(std::vector<double>{1, 2});  // $2 \times 1$
casadi::DM y0 = casadi::DM(std::vector<double>{3, 4});  // $2 \times 1$

std::vector<casadi::DM> out = F(std::vector<casadi::DM>{x0, y0});
std::cout << "out.size=" << out.size() << "\n";
std::cout << "f=" << out.at(0) << "\n";
```

常见错误（非常常见）：
- **维度不匹配**：例如 $x$ 是 $2\times 1$，你传了 $1\times 2$ 或标量，会在运行时报错。
- **顺序传错**：`{x0, y0}` 写成 `{y0, x0}` 不会编译报错，但结果含义完全变了。

### 5.4 调用方式 B：`DMDict`（按名字传参，返回 DMDict）

当你使用 `DMDict` 调用时：
- 输入：用键（字符串）对应 input 名字（如 `"x"`, `"y"`）
- 输出：返回 `DMDict`，键是 output 名字（如 `"f"`）

```cpp
casadi::DMDict arg;
arg["x"] = x0;
arg["y"] = y0;

casadi::DMDict res = F(arg);
std::cout << "f=" << res.at("f") << "\n";
```


## 6. 自动微分（AD）：jacobian / gradient / hessian / jtimes

CasADi 的 AD 在 C++ 下接口非常直接，核心是这些自由函数：
- `casadi::jacobian(f, x)`
- `casadi::gradient(f, x)`（通常 f 为标量）
- `casadi::hessian(f, x)` 返回 (H, g) 或按版本不同 API（建议先看官方 docs 对应版本）
- `casadi::jtimes(f, x, v)`（Jacobian-vector product）

示例：对一个标量目标做梯度：
```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::MX x = casadi::MX::sym("x", 2, 1);
  casadi::MX f = casadi::pow(x(0), 2) + casadi::sin(x(1));
  casadi::MX g = casadi::gradient(f, x);
  casadi::Function G("G", {x}, {f, g}, {"x"}, {"f", "g"});

  casadi::DMDict arg;
  arg["x"] = casadi::DM(std::vector<double>{3.0, 0.1});
  auto res = G(arg);
  std::cout << "f=" << res.at("f") << "\n";
  std::cout << "g=" << res.at("g") << "\n";
  return 0;
}
```

## 6.x 雅可比 / 梯度 / Hessian 小结（要点）

- **雅可比（Jacobian）**
  - 定义：对向量值函数 `f: R^n -> R^m`，雅可比矩阵 `J = ∂f/∂x`，形状为 `m x n`。元素为 `J_{i,j} = ∂f_i/∂x_j`。
  - CasADi C++：`MX J = jacobian(f, x);`。注意：若 `f` 是标量，`jacobian(f, x)` 返回 `1 x n`（行向量）。

- **梯度（Gradient）**
  - 定义：对标量函数 `f: R^n -> R`，梯度 `g = ∂f/∂x`，常表示为列向量 `n x 1`。
  - CasADi C++：`MX g = gradient(f, x);`。等价关系：`gradient(f,x) == jacobian(f,x).T()`（当 `f` 为标量时）。

- **Hessian（海森矩阵）**
  - 定义：对标量 `f`，Hessian `H = ∂^2 f / ∂x^2`，形状 `n x n`，通常是对称的。
  - CasADi C++：`MX H = hessian(f, x, g);`（`g` 可用来接收梯度）。
  - 对组合变量 `z = [x; u]`，`H` 的形状为 `(nx+nu) x (nx+nu)`，可用 `Slice` 分块为 `Hxx, Hxu, Hux, Huu`（详见第 11 节示例）。

- **高效技巧**
  - 若只需 `J·v`（Jacobian 与向量乘积），优先用 `jtimes(f, x, v)`，避免显式构造 `J`。
  - 对标量代价建议直接用 `gradient` / `hessian` 接口，语义更清晰，返回形状也更直观。
  - 注意变量拼接顺序（例如 `z = [x; u]`），梯度/切片的索引要与拼接顺序一致。

- **常见代码示例**：
```cpp
// f: R^n -> R^m
MX J = jacobian(f, x);       // m x n

// f: R^n -> R (标量)
MX g = gradient(f, x);       // n x 1
MX H = hessian(f, x, g);     // n x n

// z = [x; u] 的 Hessian 切块
MX Hfull = hessian(l, z, gz);
MX Hxx = Hfull(Slice(0, nx), Slice(0, nx));
MX Hxu = Hfull(Slice(0, nx), Slice(nx, nx + nu));
MX Hux = Hfull(Slice(nx, nx + nu), Slice(0, nx));
MX Huu = Hfull(Slice(nx, nx + nu), Slice(nx, nx + nu));

// Jacobian-vector product（高效）
MX jv = jtimes(f, x, v);
```

---


### 6.x.1 关于 `hessian` 返回的梯度 与 `gradient`/`jacobian` 的异同（补充）

- **适用场景**：
  - `gradient(f, x)`：仅对**标量**函数 `f: R^n -> R` 有意义，返回 **列向量**（`n x 1`）——`∂f/∂x`。
  - `jacobian(f, x)`：对**任意**向量值函数 `f: R^n -> R^m` 都适用，返回 `m x n` 矩阵。若 `f` 为标量，`jacobian(f,x)` 返回 `1 x n`（行向量）。
  - `hessian(f, x, g)`：一般用于**标量**函数，返回 Hessian 矩阵 `H = ∂^2 f / ∂x^2`（`n x n`），并可同时得到梯度 `g`（`n x 1`）。

- **数值/形状对比**：
  - 标量 `f` 时：
    - `gradient(f,x)` 的结果等于 `jacobian(f,x).T()`（二者值完全相同，但形状不同）。
    - `hessian(f,x,g)` 返回的 `g`（若以引用/输出参数形式接收）与 `gradient(f,x)` 值相同且通常为列向量（`n x 1`）。

- **性能与实现建议**：
  - 若只需要梯度：用 `gradient(f,x)`（更语义化、通常更省计算）。
  - 若需要 Hessian：直接调用 `hessian(f,x,g)` 更方便且通常更高效，因为计算 Hessian 时会重用中间导数结果并同时提供梯度 `g`。
  - 若只需 `J·v`（Jacobian 与向量乘积）：优先用 `jtimes(f, x, v)`，比构造完整 `J` 更高效。

- **示例代码**：
```cpp
// 标量 f
MX g1 = gradient(f, x);        // n x 1
MX j  = jacobian(f, x);         // 1 x n
MX g2 = j.T();                  // n x 1 (等于 g1)

// Hessian 同时返回梯度
MX g; MX H = hessian(f, x, g);  // H: n x n, g: n x 1, g == g1
```

---


## 7. integrator：常微分方程/DAE 积分（cvodes/idas）

integrator 在 C++ 下通常按“构造一个字典 + 调用”来用：
- 构造 `dae`（键如 `x`, `p`, `ode`，有时还有 `z`, `alg`）
- `casadi::integrator(name, plugin, dae, opts)`
- 调用后得到 `DMDict`（常见输出 `xf`, `qf` 等）

示例：$\dot x = -x + u$，从 $t=0$ 积到 $t=1$：
```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::MX x = casadi::MX::sym("x");
  casadi::MX u = casadi::MX::sym("u");
  casadi::MX ode = -x + u;

  casadi::Dict dae;
  dae["x"] = x;
  dae["p"] = u;
  dae["ode"] = ode;

  casadi::Dict opts;
  opts["tf"] = 1.0;

  casadi::Function I = casadi::integrator("I", "cvodes", dae, opts);

  casadi::DMDict arg;
  arg["x0"] = 0.0;
  arg["p"] = 1.0;
  casadi::DMDict res = I(arg);
  std::cout << "xf=" << res.at("xf") << "\n";
  return 0;
}
```

如果你的系统安装里没有 `cvodes` 插件，可以把插件名换成你可用的（或先排查插件加载问题）。

## 8. nlpsol（IPOPT）：用 C++ 构建并求解 NLP

最常见流程：
1) 构建决策变量 `x`、目标 `f`、约束 `g`（通常用 `MX`）
2) `nlpsol("solver", "ipopt", nlp, opts)`
3) 用 `DMDict` 给出 `lbx/ubx/lbg/ubg/x0`
4) `res.at("x") / res.at("f") / res.at("lam_g") ...` 取解

示例：
$$\min_x\; x_0^2 + x_1^2\quad s.t.\; x_0 + x_1 = 10$$

```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::MX x = casadi::MX::sym("x", 2, 1);
  casadi::MX f = casadi::pow(x(0), 2) + casadi::pow(x(1), 2);
  casadi::MX g = x(0) + x(1) - 10;

  casadi::Dict nlp;
  nlp["x"] = x;
  nlp["f"] = f;
  nlp["g"] = g;

  casadi::Dict opts;
  opts["ipopt.print_level"] = 0;
  opts["print_time"] = 0;

  casadi::Function solver = casadi::nlpsol("solver", "ipopt", nlp, opts);

  casadi::DMDict arg;
  arg["lbx"] = -casadi::DM::inf();
  arg["ubx"] =  casadi::DM::inf();
  arg["lbg"] = 0;
  arg["ubg"] = 0;
  arg["x0"] = casadi::DM(std::vector<double>{0.0, 0.0});

  casadi::DMDict res = solver(arg);
  std::cout << "x*=" << res.at("x") << "\n";
  std::cout << "f*=" << res.at("f") << "\n";
  return 0;
}
```

如果报 `Plugin 'ipopt' is not found`，先回到第 1 章检查插件加载。

## 9. Opti：用更“建模”的方式写优化问题（C++）

`casadi::Opti` 是 CasADi 提供的建模层：变量、约束、目标更像数学表达式。C++ 下用法与概念和官方示例一致。

### 9.1 最小例子：带等式约束的二次目标
```cpp
#include <casadi/casadi.hpp>

int main() {
  casadi::Opti opti;

  casadi::MX x = opti.variable(2, 1);
  casadi::MX f = casadi::pow(x(0),2) + casadi::pow(x(1),2);
  opti.minimize(f);
  opti.subject_to(x(0) + x(1) == 10);

  opti.solver("ipopt", casadi::Dict{{"print_time", 0}, {"ipopt.print_level", 0}});
  casadi::OptiSol sol = opti.solve();
  std::cout << "x*=" << sol.value(x) << "\n";
  std::cout << "f*=" << sol.value(f) << "\n";
  return 0;
}
```

### 9.2 参数与 warm-start（MPC 常用）
- 把会变的量建成 `parameter()`
- 每次求解前更新参数值
- 用 `set_initial` 给变量一个好的初值（上一时刻解）

注意：Opti 的具体 warm-start 深度配置（比如传入 `ipopt.warm_start_init_point`）仍然走 solver options（`Dict`）。